In [9]:
import pandas as pd
df = pd.read_csv("small_df.csv",delimiter=',')
feature_df = pd.read_csv("tfidf_matrix.csv",delimiter=',')

In [10]:
feature_df.drop(columns=['Unnamed: 0'], inplace=True)

In [11]:
# load the URL data
url_dict = pd.read_csv("restaurant_url.csv", header=None, index_col=0)
url_dict = url_dict[1].to_dict()
# delete the header
del url_dict['Restaurant']


In [12]:
#load summary
import csv
with open("Summaries_of_restaurants.csv", "r", newline="") as csvfile:
  reader = csv.reader(csvfile)
  summary = {}
  summ = []
  for row in reader:
    summ.append(row[1])
  i=1
  for key in df:
    summary[key] = summ[i]
    i = i+1

In [13]:
def initilizing():
    corpus = []#all possible valid input words 
    for key in feature_df:
        corpus.append(key)

    return corpus

In [14]:
#rank all restaurants, get their scores, which store in 'ranks'
#input:a list of string(words from all filters)
#output:a dictionary, value is the score of restaurant
def ranking(filter_words, corpus):
    ranks = {}
    for key in df:
        ranks[key] = 0
    n = 0
    for word in filter_words: 
        if(word in corpus): 
            i = 0
            for key in ranks:
                ranks[key] = (n*ranks[key]+feature_df[word][i])/(n+1)
                i = i+1
        n = n+1
    return ranks

In [15]:
#used by submit and next button
#input:next_time is default when first time called , and a list of four filter words
#next_time is the time that next button is pressed
#output:a string(name of a restaurant)
def submitting_and_next(filter_words=[], next_time=0):
    corpus = initilizing()
    ranks = ranking(filter_words, corpus)
    #pick the (next_times+1)th best restaurant
    for i in range(next_time+1):
        recommend = "十六區和風料理"
        highest_score = ranks["十六區和風料理"]
        for key in ranks:
            if (ranks[key] > highest_score):
                recommend = key
                highest_score = ranks[key]
        ranks[recommend] = 0

    if(highest_score == 0):
        return {"Restaurant":"No other recommendation","Summary":"NULL"}
    return {"Restaurant":recommend,"Summary":summary[recommend], "URL":url_dict[recommend]}

## DEMO

In [16]:
filter_words = ["flavor","beef"]

In [17]:
submitting_and_next(filter_words)

{'Restaurant': '如意麵食館 牛肉麵',
 'Summary': 'Pumpkin beef soup is delicious,noodle itself was just ok,not special. Peanut beef roll was also tasty,especially skin was the best in Hsinchu,with peanut butter inside. Not the best beef noodle Ive had, but definitely the most creative. They also sell different soup base beef noodles, like tomato and pumpkin.',
 'URL': 'https://www.google.com/maps/place/%E5%A6%82%E6%84%8F%E9%BA%B5%E9%A3%9F%E9%A4%A8+%E7%89%9B%E8%82%89%E9%BA%B5/@24.8007444,120.9860451,17z/data=!3m1!4b1!4m6!3m5!1s0x346835e05eb4914d:0xfd9c8ff9144e656!8m2!3d24.8007396!4d120.98862!16s%2Fg%2F1tfrdhky?entry=ttu'}

In [18]:
# first time use next
submitting_and_next(filter_words,1)

{'Restaurant': 'Shi Tzi Beef Noodles',
 'Summary': 'The beef noodles were good and had a decent amount of tender meat for 120NT. The store is known for their rolls though, so make sure to try those! Although a little bit dry, they were full of flavor and very tasty. On a Friday around 6:30 there was around a ten minute wait.',
 'URL': 'https://www.google.com/maps/place/%E7%92%BD%E5%AD%90%E7%89%9B%E8%82%89%E9%BA%B5/@24.7971102,120.9847323,16.71z/data=!4m6!3m5!1s0x346835e05baa2671:0xb8398ae101d6085!8m2!3d24.7974231!4d120.985294!16s%2Fg%2F1tzvt8yq?entry=ttu'}

In [19]:
# second time use next
for i in range(2,5):
    print(submitting_and_next(filter_words,i))


{'Restaurant': 'Duan Chun Zhen', 'Summary': 'I have the beef noodle here.The taste is really nice, serve with a big chunk of beef.The soup is a bit too salty but overall I still love the taste here. The price is pretty expensive compared to other shops. The beef noodles of Dian Chin Zhen must be enjoyed here in Hsinchu.', 'URL': 'https://www.google.com/maps/place/%E6%AE%B5%E7%B4%94%E8%B2%9E%E5%BB%BA%E5%8A%9F%E5%BA%97/@24.8023115,120.9923741,18.29z/data=!4m6!3m5!1s0x34683671561b906d:0x89d51fd0d5b254a2!8m2!3d24.8026457!4d120.9937853!16s%2Fg%2F1tx44s4c?entry=ttu'}
{'Restaurant': '八分飽餐廳', 'Summary': 'Good solid food. Clean tasting with subtle flavors for my Western palette. My favorite was the sour vegetables with beef wraps. Free all you can eat ice cream. They offer business lunch for $100+ per dish plus service charge. If you are looking for a bugget Chinese style meal, this is an good option.', 'URL': 'https://www.google.com/maps/search/%E5%85%AB%E5%88%86%E9%A3%BD%E9%A4%90%E5%BB%B3/@24